# Read & Clear Data

In [3]:
import numpy as np
import pandas as pd
import feather 
import matplotlib.pyplot as plt  
from time import time
from mailerWithUtf8 import mail

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split  
from sklearn.externals import joblib
from sklearn import preprocessing
import os

In [4]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
#     df.to_csv(out_filename + ".csv", encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [5]:
start_time = time()
path = "C:/Users/VIPLAB/Desktop/preprocess_py/marketing_analyze/"
# filename = '0713_april_marketing_anylze_full_table_group12_max_min_train.feather' 
filename = '0713_april_marketing_anylze_full_table_group12_select_max_min_train.feather' 

train_df = feather.read_dataframe(path + filename)
# df = pd.read_csv(path + filename, error_bad_lines=False, nrows = 10)

print('load feather %s seconds', format(time() - start_time))


load feather %s seconds 0.4730348587036133


In [6]:
# filename = '0713_april_marketing_anylze_full_table_group12_max_min_test.feather' 
filename = '0713_april_marketing_anylze_full_table_group12_select_max_min_test.feather' 
# filename = '0713_april_marketing_anylze_full_table_group12_max_min.feather' 

test_df = feather.read_dataframe(path + filename)
print('load feather %s seconds', format(time() - start_time))

load feather %s seconds 0.6090028285980225


In [7]:
CLASSIFIER = "DECISION"
CURRENT_MODE = "SELECT_MODE"
groups_idx = 10

In [10]:
groups = train_df["Groups"].unique()

In [9]:
target_groups = groups[groups_idx]

In [12]:
out_path = "clf_random_forest_model/all/" 
if not os.path.exists(out_path):
    os.makedirs(out_path)

In [ ]:
out_path = "clf_random_forest_model/" + target_groups + "/" 
if not os.path.exists(out_path):
    os.makedirs(out_path)

# for each group build tree

In [ ]:
map_target_groups = []
for group in groups:
    if(group == target_groups):
        map_target_groups.append(group)
    else:
        map_target_groups.append("None")
d = dict(zip(groups, map_target_groups))
train_df["Groups"] = train_df["Groups"].map(d)
test_df["Groups"] = test_df["Groups"].map(d)

In [ ]:
train_df.groupby("Groups").count()

In [ ]:
# test_class_count = test_df.groupby("Groups").count()[test_df.columns[0:1]].values.flatten()

In [13]:
train_x, train_y = train_df.iloc[:, 0:-1].values, train_df.iloc[:, -1].values

In [14]:
test_x, test_y = test_df.iloc[:, 0:-1].values, test_df.iloc[:, -1].values

# convert y to numeric# 

In [15]:
le = preprocessing.LabelEncoder()
le.fit(train_df["Groups"].unique())
train_numeric_y = le.transform(train_y)
test_numeric_y = le.transform(test_y) 

# build Random Forest Classifier


In [16]:
features = train_df.columns[0:4]

In [ ]:
clf = RandomForestClassifier(n_jobs=5,n_estimators=50)

In [ ]:
start_time = time()
clf.fit(train_x, train_y)


# saving classifier

In [ ]:
joblib.dump(clf, out_path + out_filename + 'CLF.pkl') 
# clf = joblib.load('filename.pkl') 


In [ ]:
clf_info = str(("Accuracy on training set: %f" % clf.score(train_x, train_numeric_y))) + '\n'
clf_info += str(("Accuracy on test set: %f" % clf.score(test_x, test_numeric_y))) + '\n'
clf_info += str(('fit time %s seconds' % format(time() - start_time))) + '\n'
print(clf_info)

In [ ]:
clf.predict(test_x)

In [ ]:
test_numeric_y

# get feature importances

In [ ]:
important_dict = dict(zip(train_df.columns[:-1], clf.feature_importances_))

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')

In [9]:
wants_cols = ['SRV_TYPE_DESC',
 'GENDER_CODE',
 'AGE',
 'TENURE_SCV',
 'BILL_CITY_NAME',
 'DORMANT_MONTH_CNT_MKT',
 'IMEI_MFG_NAME',
 'IMEI_BAND',
 'CHURN_CLM_IND',
 'PTY_NP_IN_IND',
 'PTY_NP_OUT_IND',
 'MDS_CURR_RENEW_CNT',
 'PROM_CURR_EXP_MONTH_CNT',
 'CURR_PROM_COMMIT_MONTHS',
 'L1M_PROM_COMMIT_MONTHS',
 'NET_INV_AMT',
 'L1M_NET_INV_AMT',
 'L2M_NET_INV_AMT',
 'L3M_NET_INV_AMT',
 'AGENCY_AMT',
 'L3M_AVG_NET_INV_AMT',
 'MT_STM_AMT',
 'VOICE_RC_AMT',
 'DATA_RC_AMT',
 'ONNET_AMT',
 'OFFNETM_AMT',
 'PSTN_AMT',
 'INT_ROAM_AMT',
 'IDD_AMT',
 'DATA_INV_AMT',
 'VOICE_INV_AMT',
 'GOOGLE_PLAY_IND',
 'MO_TTL_DURATION',
 'MO_ONNET_DUR',
 'MO_OFFNET_DUR',
 'PSTN_DUR',
 'MO_TTL_COUNT',
 'MO_ONNET_CNT',
 'MO_OFFNET_CNT',
 'PSTN_CNT',
 'MT_TTL_DURATION',
 'MT_ONNET_DUR',
 'MT_OFFNET_DUR',
 'MT_PSTN_DUR',
 'MT_TTL_COUNT',
 'MT_ONNET_CNT',
 'MT_OFFNET_CNT',
 'MT_PSTN_CNT',
 'MOST_MO_OPERATOR',
 'MOST_MT_OPERATOR',
 'MOC_FET_DUR',
 'MOC_TWM_DUR',
 'MOC_CHM_DUR',
 'MOC_OTH_DUR',
 'MOC_PSTN_DUR',
 'MOST_MO_DUR',
 'MTC_FET_DUR',
 'MTC_TWM_DUR',
 'MTC_CHM_DUR',
 'MTC_PSTN_DUR',
 'MOST_MT_DUR',
 'CUST_TYPE',
 'FLAT_RATE_4G_IND',
 'FLAT_RATE_3G_IND',
 'VOICE_MONTHLY_FEE',
 'DATA_MONTHLY_FEE',
 'TOTAL_MONTHLY_FEE',
 'DATA_RATING_USAGE_MB',
 'HS_CHG_CNT',
 'AVG_HS_USE_MONTH',
 'Groups']


In [12]:
df.head()

,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,DORMANT_MONTH_CNT_MKT,IMEI_MFG_NAME,IMEI_BAND,CHURN_CLM_IND,PTY_NP_IN_IND,...,CUST_TYPE,FLAT_RATE_4G_IND,FLAT_RATE_3G_IND,VOICE_MONTHLY_FEE,DATA_MONTHLY_FEE,TOTAL_MONTHLY_FEE,DATA_RATING_USAGE_MB,HS_CHG_CNT,AVG_HS_USE_MONTH,Groups
0,0.0,0.0,0.000000,0.076014,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.444239,0.441621,0.005087,0.000000,0.000000,InstantMessage-Low
1,0.0,0.0,0.005495,0.941723,0.045455,0.0,0.001170,0.0,0.5,0.0,...,0.0,0.0,0.0,0.000000,0.221934,0.220626,0.000184,0.066667,0.005988,Infrequent_User
2,0.0,0.5,0.010989,0.016892,0.090909,0.0,0.002339,0.0,0.5,1.0,...,0.0,0.0,0.0,0.000000,0.518340,0.515285,0.007588,0.133333,0.011976,No_ignificant_preference
3,1.0,0.0,0.016484,0.089105,0.136364,0.0,0.003509,0.0,0.5,1.0,...,0.0,1.0,1.0,0.225496,0.285291,0.430203,0.008652,0.000000,0.017964,No_ignificant_preference
4,0.0,0.5,0.016484,0.058699,0.045455,0.0,0.000000,0.0,0.5,1.0,...,0.0,0.0,0.0,0.000000,0.296036,0.294291,0.003747,0.133333,0.011976,InstantMessage-Low


In [13]:
len(df)

1884095

In [14]:
df = df[df["Groups"] != "None"]

In [15]:
len(df)

1884095

In [16]:
# verify deletion
df.groupby('Groups').count()

,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,DORMANT_MONTH_CNT_MKT,IMEI_MFG_NAME,IMEI_BAND,CHURN_CLM_IND,PTY_NP_IN_IND,...,MOST_MT_DUR,CUST_TYPE,FLAT_RATE_4G_IND,FLAT_RATE_3G_IND,VOICE_MONTHLY_FEE,DATA_MONTHLY_FEE,TOTAL_MONTHLY_FEE,DATA_RATING_USAGE_MB,HS_CHG_CNT,AVG_HS_USE_MONTH
Groups,,,,,,,,,,,,,,,,,,,,,
Adult,54309,54309,54309,54309,54309,54309,54309,54309,54309,54309,...,54309,54309,54309,54309,54309,54309,54309,54309,54309,54309
Game,87618,87618,87618,87618,87618,87618,87618,87618,87618,87618,...,87618,87618,87618,87618,87618,87618,87618,87618,87618,87618
HomeLife,74949,74949,74949,74949,74949,74949,74949,74949,74949,74949,...,74949,74949,74949,74949,74949,74949,74949,74949,74949,74949
Infrequent_User,145107,145107,145107,145107,145107,145107,145107,145107,145107,145107,...,145107,145107,145107,145107,145107,145107,145107,145107,145107,145107
InstantMessage-High,424699,424699,424699,424699,424699,424699,424699,424699,424699,424699,...,424699,424699,424699,424699,424699,424699,424699,424699,424699,424699
InstantMessage-Low,431903,431903,431903,431903,431903,431903,431903,431903,431903,431903,...,431903,431903,431903,431903,431903,431903,431903,431903,431903,431903
Map,103786,103786,103786,103786,103786,103786,103786,103786,103786,103786,...,103786,103786,103786,103786,103786,103786,103786,103786,103786,103786
News,57174,57174,57174,57174,57174,57174,57174,57174,57174,57174,...,57174,57174,57174,57174,57174,57174,57174,57174,57174,57174
No_ignificant_preference,266966,266966,266966,266966,266966,266966,266966,266966,266966,266966,...,266966,266966,266966,266966,266966,266966,266966,266966,266966,266966


# Split Data

In [17]:
# train, test = train_test_split(df, test_size = 0.4, random_state = 200)
train, test = train_test_split(df, test_size = 0.33, random_state = 200)

In [18]:
# train_x, train_y = train.iloc[:, 0:-1], train.iloc[:, -1]

In [19]:
# test_x, test_y = test.iloc[:, 0:-1], test.iloc[:, -1]

In [20]:
len(train)

1507276

In [21]:
len(test)

376819

In [22]:
train.groupby('Groups').count()

,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,DORMANT_MONTH_CNT_MKT,IMEI_MFG_NAME,IMEI_BAND,CHURN_CLM_IND,PTY_NP_IN_IND,...,MOST_MT_DUR,CUST_TYPE,FLAT_RATE_4G_IND,FLAT_RATE_3G_IND,VOICE_MONTHLY_FEE,DATA_MONTHLY_FEE,TOTAL_MONTHLY_FEE,DATA_RATING_USAGE_MB,HS_CHG_CNT,AVG_HS_USE_MONTH
Groups,,,,,,,,,,,,,,,,,,,,,
Adult,43512,43512,43512,43512,43512,43512,43512,43512,43512,43512,...,43512,43512,43512,43512,43512,43512,43512,43512,43512,43512
Game,70057,70057,70057,70057,70057,70057,70057,70057,70057,70057,...,70057,70057,70057,70057,70057,70057,70057,70057,70057,70057
HomeLife,59981,59981,59981,59981,59981,59981,59981,59981,59981,59981,...,59981,59981,59981,59981,59981,59981,59981,59981,59981,59981
Infrequent_User,116011,116011,116011,116011,116011,116011,116011,116011,116011,116011,...,116011,116011,116011,116011,116011,116011,116011,116011,116011,116011
InstantMessage-High,339830,339830,339830,339830,339830,339830,339830,339830,339830,339830,...,339830,339830,339830,339830,339830,339830,339830,339830,339830,339830
InstantMessage-Low,345491,345491,345491,345491,345491,345491,345491,345491,345491,345491,...,345491,345491,345491,345491,345491,345491,345491,345491,345491,345491
Map,83009,83009,83009,83009,83009,83009,83009,83009,83009,83009,...,83009,83009,83009,83009,83009,83009,83009,83009,83009,83009
News,45705,45705,45705,45705,45705,45705,45705,45705,45705,45705,...,45705,45705,45705,45705,45705,45705,45705,45705,45705,45705
No_ignificant_preference,213444,213444,213444,213444,213444,213444,213444,213444,213444,213444,...,213444,213444,213444,213444,213444,213444,213444,213444,213444,213444


In [23]:
test.groupby('Groups').count()

,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,DORMANT_MONTH_CNT_MKT,IMEI_MFG_NAME,IMEI_BAND,CHURN_CLM_IND,PTY_NP_IN_IND,...,MOST_MT_DUR,CUST_TYPE,FLAT_RATE_4G_IND,FLAT_RATE_3G_IND,VOICE_MONTHLY_FEE,DATA_MONTHLY_FEE,TOTAL_MONTHLY_FEE,DATA_RATING_USAGE_MB,HS_CHG_CNT,AVG_HS_USE_MONTH
Groups,,,,,,,,,,,,,,,,,,,,,
Adult,10797,10797,10797,10797,10797,10797,10797,10797,10797,10797,...,10797,10797,10797,10797,10797,10797,10797,10797,10797,10797
Game,17561,17561,17561,17561,17561,17561,17561,17561,17561,17561,...,17561,17561,17561,17561,17561,17561,17561,17561,17561,17561
HomeLife,14968,14968,14968,14968,14968,14968,14968,14968,14968,14968,...,14968,14968,14968,14968,14968,14968,14968,14968,14968,14968
Infrequent_User,29096,29096,29096,29096,29096,29096,29096,29096,29096,29096,...,29096,29096,29096,29096,29096,29096,29096,29096,29096,29096
InstantMessage-High,84869,84869,84869,84869,84869,84869,84869,84869,84869,84869,...,84869,84869,84869,84869,84869,84869,84869,84869,84869,84869
InstantMessage-Low,86412,86412,86412,86412,86412,86412,86412,86412,86412,86412,...,86412,86412,86412,86412,86412,86412,86412,86412,86412,86412
Map,20777,20777,20777,20777,20777,20777,20777,20777,20777,20777,...,20777,20777,20777,20777,20777,20777,20777,20777,20777,20777
News,11469,11469,11469,11469,11469,11469,11469,11469,11469,11469,...,11469,11469,11469,11469,11469,11469,11469,11469,11469,11469
No_ignificant_preference,53522,53522,53522,53522,53522,53522,53522,53522,53522,53522,...,53522,53522,53522,53522,53522,53522,53522,53522,53522,53522


In [24]:
train.head()

,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,DORMANT_MONTH_CNT_MKT,IMEI_MFG_NAME,IMEI_BAND,CHURN_CLM_IND,PTY_NP_IN_IND,...,CUST_TYPE,FLAT_RATE_4G_IND,FLAT_RATE_3G_IND,VOICE_MONTHLY_FEE,DATA_MONTHLY_FEE,TOTAL_MONTHLY_FEE,DATA_RATING_USAGE_MB,HS_CHG_CNT,AVG_HS_USE_MONTH,Groups
1834895,0.0,0.0,0.637363,0.045186,0.454545,0.0,0.003509,0.000000,0.5,1.0,...,0.0,0.0,0.0,0.0,0.221934,0.220626,0.001610,0.133333,0.011976,Map
1598361,0.0,0.0,0.181319,0.135980,0.681818,0.0,0.003509,0.000000,0.5,0.0,...,0.0,0.0,0.0,0.0,0.296036,0.294291,0.007767,0.200000,0.107784,HomeLife
1816889,0.0,0.5,0.060440,0.729730,0.090909,0.0,0.000000,0.333333,1.0,0.0,...,0.0,0.0,0.0,0.0,0.666543,0.662615,0.002039,0.266667,0.041916,InstantMessage-Low
1515765,0.0,0.5,0.291209,0.078970,0.318182,0.0,0.000000,0.000000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.370137,0.367956,0.001623,0.000000,0.035928,InstantMessage-Low
1381281,0.0,0.5,0.071429,0.369088,0.045455,0.0,0.003509,0.000000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.370137,0.367956,0.002107,0.200000,0.023952,InstantMessage-High


In [52]:
path = "./marketing_analyze/"
# path = "C:/Users/VIPLAB/Downloads/Spark_data/"
save_dataframe(train, path + out_filename + "_train")

time for output csv file: 120.78


In [ ]:
save_dataframe(test, path + out_filename + "_test")

In [ ]:
train_sample = train.sample(frac=0.1)
len(train_sample)
save_dataframe(train_sample, path + out_filename + "_train_sample")

In [ ]:
test_sample = test.sample(frac=0.1)
len(test_sample)
save_dataframe(test_sample, path + out_filename+ "_test_sample")

In [ ]:
print(len(train_sample))
print(len(test_sample))